In [1]:
import pandas as pd
import os

In [28]:
non_param_cusum_path = os.path.join(os.getcwd(), 'non_parametric_cusum')
nonparam_cusum_ret = pd.read_csv(os.path.join(non_param_cusum_path, 'nonp_cusum_results_spx_ret.csv'), index_col=0, parse_dates=True)
nonparam_cusum_vol = pd.read_csv(os.path.join(non_param_cusum_path, 'nonp_cusum_results_spx_vol.csv'), index_col=0, parse_dates=True)

nonparam_cusum_results = pd.merge(nonparam_cusum_ret, nonparam_cusum_vol, how='inner', left_index=True, right_index=True, suffixes=('_n_cusum_ret', '_n_cusum_vol'))

In [38]:
nonparam_cusum_results.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7466 entries, 1995-01-04 to 2024-08-29
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   ret_flag_n_cusum_ret  7466 non-null   int64
 1   vol_flag_n_cusum_ret  7466 non-null   int64
 2   ret_flag_n_cusum_vol  7466 non-null   int64
 3   vol_flag_n_cusum_vol  7466 non-null   int64
dtypes: int64(4)
memory usage: 549.7 KB


In [35]:
pelt_path = os.path.join(os.getcwd(), 'pelt')
pelt_ret = pd.read_csv(os.path.join(pelt_path, 'pelt_results_spx.csv'), index_col=1, parse_dates=True)
pelt_ret = pelt_ret.drop(columns=pelt_ret.columns[0:2]) # drop redundant columns including original returns
pelt_ret.columns = ['pelt_cp_flag']

In [37]:
pelt_ret.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7466 entries, 1995-01-04 to 2024-08-29
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   pelt_cp_flag  7466 non-null   int64
dtypes: int64(1)
memory usage: 116.7 KB


In [41]:
wbs_path = os.path.join(os.getcwd(), 'wbs')
wbs_ret = pd.read_csv(os.path.join(wbs_path, 'cp_wbs_ret.csv'), index_col=0, parse_dates=True)
wbs_vol = pd.read_csv(os.path.join(wbs_path, 'cp_wbs_vol.csv'), index_col=0, parse_dates=True)

wbs_results = pd.merge(wbs_ret, wbs_vol, how='inner', left_index=True, right_index=True, suffixes=('_wbs_ret', '_wbs_vol'))

In [42]:
wbs_results.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7467 entries, 1995-01-04 to 2024-08-30
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ChangePoint_wbs_ret  7467 non-null   int64
 1   ChangePoint_wbs_vol  7467 non-null   int64
dtypes: int64(2)
memory usage: 175.0 KB


In [53]:
cp_flags = pd.concat([nonparam_cusum_results, pelt_ret, wbs_results], axis=1)
cp_flags = cp_flags.resample('M').apply(lambda x: x.iloc[-1]).ffill() # fill last row of data
cp_flags.index = cp_flags.index.to_period('M').to_timestamp() # change the date to BOM

In [54]:
cp_flags.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 356 entries, 1995-01-01 to 2024-08-01
Freq: MS
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ret_flag_n_cusum_ret  356 non-null    float64
 1   vol_flag_n_cusum_ret  356 non-null    float64
 2   ret_flag_n_cusum_vol  356 non-null    float64
 3   vol_flag_n_cusum_vol  356 non-null    float64
 4   pelt_cp_flag          356 non-null    float64
 5   ChangePoint_wbs_ret   356 non-null    int64  
 6   ChangePoint_wbs_vol   356 non-null    int64  
dtypes: float64(5), int64(2)
memory usage: 22.2 KB


In [55]:
cp_flags.to_csv('changepoint_flags.csv', index=True)